In [1]:
# Include main project path in the system path for python
import sys
import os
import re
import pandas as pd
import urllib
from tabulate import tabulate
import timeit
from tqdm import tqdm_notebook as tqdm
import re
sys.path.insert( 0,os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import the breadboard module
from breadboard import BreadboardClient

## Get parameters for images

In [2]:
# initialize the client
bc = BreadboardClient(config_path='API_CONFIG.json', lab_name='bec1')

In [5]:
imagenames = ['10-09-2018_00_27_44_TopA','10-09-2018_00_27_44_TopB']
params = ['FB_field_13_V','ImagFreq0']
# params = "list_bound_only"

# df = bc.get_images_df(imagenames)
df = bc.get_images_df(imagenames, params)
# df = bc.get_images_df(imagenames, 'FB_field_13_V')

In [6]:
df.head()

,imagename,x,FB_field_13_V,unixtime,ImagFreq0
0,10-09-2018_00_27_44_TopA,1.539045e+09,3.774,1.539045e+09,193.95
1,10-09-2018_00_27_44_TopB,1.539045e+09,3.774,1.539045e+09,193.95


In [7]:
tabulate(df, tablefmt="pipe", headers="keys")

'|    | imagename                |           x |   FB_field_13_V |    unixtime |   ImagFreq0 |\n|---:|:-------------------------|------------:|----------------:|------------:|------------:|\n|  0 | 10-09-2018_00_27_44_TopA | 1.53904e+09 |           3.774 | 1.53904e+09 |      193.95 |\n|  1 | 10-09-2018_00_27_44_TopB | 1.53904e+09 |           3.774 | 1.53904e+09 |      193.95 |'

note: the API does some caching so the data might be a little outdated

## Update notes for a single image

In [6]:
params_update  = {
            'notes': "test",
            'pixel_size': 2
        }

In [7]:
resp = bc.update_image( df.at[0,'id'],df.at[0,'imagename'], params_update)

In [8]:
resp.status_code

200

## Fermi 3 tests

In [8]:
# initialize the client
bc = BreadboardClient(config_path='API_CONFIG.json', debug=True)

DEBUG:root:Hi! You are debugging breadboard.


In [9]:
df = bc.get_images_df(['2019-02-27_20-28-25_SensicamQE','2019-02-27_20-32-42_SensicamQE'], paramsin = "list_bound_only", xvar='reWhirrFreq')
df.head()

DEBUG:root:Number of images to get:2


DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:8000 "POST /images/ HTTP/1.1" 200 5743
DEBUG:root:2019-02-27T20:32:42Z
DEBUG:root:2019-02-27T20:28:25Z


,imagename,x,TOF,accordionHoldSpacing,unixtime,reWhirrFreq,evapEndTOPMHz,holdTime
0,2019-02-27_20-28-25_SensicamQE,4.675,0.0,4.4,1.551299e+09,4.675,1777.0,10.0
1,2019-02-27_20-32-42_SensicamQE,4.725,0.0,4.4,1.551300e+09,4.725,1777.0,10.0


In [8]:
tabulate(df, tablefmt="pipe", headers="keys")

'|    | imagename                      |     x |   TOF |   accordionHoldSpacing |   reWhirrFreq |   unixtime |   holdTime |   evapEndTOPMHz |\n|---:|:-------------------------------|------:|------:|-----------------------:|--------------:|-----------:|-----------:|----------------:|\n|  0 | 2019-02-27_20-28-25_SensicamQE | 4.725 |     0 |                    4.4 |         4.725 | 1.5513e+09 |         10 |            1777 |\n|  1 | 2019-02-27_20-32-42_SensicamQE | 4.675 |     0 |                    4.4 |         4.675 | 1.5513e+09 |         10 |            1777 |'

In [10]:
df = pd.read_csv('images_rotini_ls.txt')
dfout = bc.get_images_df(df.images.tolist()[:3], paramsin=['id', 'runtime','created'], force_match=False)

DEBUG:root:Number of images to get:3


DEBUG:urllib3.connectionpool:Resetting dropped connection: localhost
DEBUG:urllib3.connectionpool:http://localhost:8000 "POST /images/ HTTP/1.1" 200 9888
DEBUG:root:2019-06-19T19:33:04Z
DEBUG:root:2019-06-19T19:32:28Z
DEBUG:root:2019-06-19T19:30:49Z


In [7]:
dfout

,imagename,x,created,id,runtime,unixtime
0,2019-06-19_19-30-46_SensicamQE.fits,1.560973e+09,2019-06-19T19:30:46Z,63585.0,2019-06-19T19:30:49Z,1.560973e+09
1,2019-06-19_19-32-26_SensicamQE.fits,1.560973e+09,2019-06-19T19:32:26Z,63586.0,2019-06-19T19:32:28Z,1.560973e+09
2,2019-06-19_19-33-01_SensicamQE.fits,1.560973e+09,2019-06-19T19:33:01Z,66193.0,2019-06-19T19:33:04Z,1.560973e+09


In [10]:
dfout.head()

,imagename,x,id,runtime,unixtime,created
0,2019-06-19_19-30-46_SensicamQE.fits,1.560973e+09,63585.0,2019-06-19T19:30:49Z,1.560973e+09,2019-06-19T19:30:46Z
1,2019-06-19_19-32-26_SensicamQE.fits,1.560973e+09,63586.0,2019-06-19T19:32:28Z,1.560973e+09,2019-06-19T19:32:26Z
2,2019-06-19_19-33-01_SensicamQE.fits,1.560973e+09,66193.0,2019-06-19T19:33:04Z,1.560973e+09,2019-06-19T19:33:01Z


In [268]:
ims = df.images.tolist()[0:2]
ims.sort()
out = bc.post_images(ims)
results = out.json().get('results')
# 63585 to 67288

In [269]:
results

[{'atom': None,
  'atomsperpixel': None,
  'bad_shot': None,
  'created': '2019-06-19T19:32:26Z',
  'cropi': None,
  'filepath': None,
  'id': 63586,
  'name': '2019-06-19_19-32-26_SensicamQE.fits',
  'notes': None,
  'odpath': None,
  'pixel_size': None,
  'run': {'created': '2019-06-19T23:32:29.026454Z',
   'dataset': None,
   'id': 115415,
   'notes': '',
   'parameters': {'2VQuadstirrPhase': 3.81,
    '4VstirrPhase': 3.846,
    'Decomp QUAD': 0.0,
    'F=2 iAmp SCell': 0.25,
    'Img_time': 3.0,
    'Initial MW power': 7.0,
    'IterationCount': 1344.0,
    'IterationNum': 0.0,
    'Li 3/2 Cell imA': 0.12,
    'Li 3/2 Cell imF': 4.5,
    'Li F=1/2 pAmp': 1.0,
    'Li F=1/2 pFreq': 4.5,
    'Li F=3/2 iAmp': 0.2,
    'Li F=3/2 iFreq': 5.826,
    'Li F=3/2 pAmp': 0.15,
    'Li F=3/2 pFreq': 6.5,
    'Li MOT freq': 4.5,
    'Li repump freq': 4.2,
    'ListBoundVariables': ['holdTime',
     'F=2 iAmp SCell',
     'evap3EndMHz',
     'freeSpinTime',
     'Omega',
     'saddleSqueezeTime'

In [255]:
[r['run']['runtime'] for r in results][-10:]

['2019-06-19T19:39:10Z',
 '2019-06-19T19:38:35Z',
 '2019-06-19T19:37:59Z',
 '2019-06-19T19:37:24Z',
 '2019-06-19T19:36:49Z',
 '2019-06-19T19:36:13Z',
 '2019-06-19T19:35:38Z',
 '2019-06-19T19:33:04Z',
 '2019-06-19T19:32:28Z',
 '2019-06-19T19:30:49Z']

In [258]:
[r['run']['runtime'] for r in results][:10]

['2019-06-19T19:39:10Z',
 '2019-06-19T19:38:35Z',
 '2019-06-19T19:37:59Z',
 '2019-06-19T19:37:24Z',
 '2019-06-19T19:36:49Z',
 '2019-06-19T19:36:13Z',
 '2019-06-19T19:35:38Z',
 '2019-06-19T19:33:04Z',
 '2019-06-19T19:32:28Z',
 '2019-06-19T19:30:49Z']

In [259]:
[r['name'] for r in results][-10:]

['2019-06-19_19-39-08_SensicamQE.fits',
 '2019-06-19_19-38-33_SensicamQE.fits',
 '2019-06-19_19-37-57_SensicamQE.fits',
 '2019-06-19_19-37-22_SensicamQE.fits',
 '2019-06-19_19-36-47_SensicamQE.fits',
 '2019-06-19_19-36-11_SensicamQE.fits',
 '2019-06-19_19-35-36_SensicamQE.fits',
 '2019-06-19_19-33-01_SensicamQE.fits',
 '2019-06-19_19-32-26_SensicamQE.fits',
 '2019-06-19_19-30-46_SensicamQE.fits']

In [260]:
[r['name'] for r in results][:10]

['2019-06-19_19-39-08_SensicamQE.fits',
 '2019-06-19_19-38-33_SensicamQE.fits',
 '2019-06-19_19-37-57_SensicamQE.fits',
 '2019-06-19_19-37-22_SensicamQE.fits',
 '2019-06-19_19-36-47_SensicamQE.fits',
 '2019-06-19_19-36-11_SensicamQE.fits',
 '2019-06-19_19-35-36_SensicamQE.fits',
 '2019-06-19_19-33-01_SensicamQE.fits',
 '2019-06-19_19-32-26_SensicamQE.fits',
 '2019-06-19_19-30-46_SensicamQE.fits']

In [4]:
tabulate(df, tablefmt="pipe", headers="keys")

'|    | imagename                      |           x |   cropi |   atom |    unixtime |   total_atoms |   tags |   pixel_size |   id | notes   |   holdTime |   TOF |   reWhirrFreq |   filepath |   evapEndTOPMHz | created              |   accordionHoldSpacing | url                                              |   reWhirrTime |\n|---:|:-------------------------------|------------:|--------:|-------:|------------:|--------------:|-------:|-------------:|-----:|:--------|-----------:|------:|--------------:|-----------:|----------------:|:---------------------|-----------------------:|:-------------------------------------------------|--------------:|\n|  0 | 2019-03-04_15-36-22_SensicamQE | 1.55171e+09 |     nan |    nan | 1.55171e+09 |           nan |    nan |            3 |  635 | test    |        300 |     0 |          4.75 |        nan |            1777 | 2019-03-04T15:36:22Z |                    4.4 | http://breadboard-215702.appspot.com/images/635/ |             7 |\n|  1 | 2019-03-

## tests on the raw json request

In [3]:
resp = bc.post_images(pd.read_clipboard(header=None)[0].tolist(), force_match=False)

In [4]:
resp.text

'{"count":1,"next":null,"previous":null,"results":[{"url":"http://breadboard-215702.appspot.com/images/52659/","id":52659,"name":"2019-06-19_18-18-45_SensicamQE","created":"2019-06-19T18:18:45Z","notes":null,"filepath":"\\\\\\\\18.62.1.253\\\\Raw Data\\\\Fermi3\\\\Images\\\\2019\\\\2019-06\\\\2019-06-19\\\\2019-06-19_18-18-45_SensicamQE.fits","tags":null,"cropi":null,"atom":null,"odpath":"odimages/Fermi3/2019/2019-06/2019-06-19/2019-06-19_18-18-45_SensicamQE.jpg","total_atoms":null,"settings":null,"atomsperpixel":null,"thumbnail":null,"run":null,"pixel_size":2.58e-07,"bad_shot":null}]}'

## Test pagination 

In [9]:
# initialize the client
bc = BreadboardClient(config_path='API_CONFIG.json', lab_name='fermi3')

In [20]:
with open('test_pagination_imagepost.txt', 'r') as f:
    out = f.readlines()
    
imagenames = [img[:-1] for img in out]

# imagenames = imagenames[0]  # test a single image
imagenames = imagenames[0:120]  # test a single page

In [14]:
# Test a direct single post query using imagenames
resp = bc.post_images(imagenames, force_match=False)
print('status_code = {}'.format(resp.status_code))
print('number of objects returned = {}'.format(len(resp.json().get('results'))))

status_code = 200
number of objects returned = 1


In [19]:
# Test the df method with force_match
df = bc.get_images_df(imagenames, force_match=True)
print('number of objects returned = {}'.format(len(df)))

Re-matching runs to images. Note: this takes some time, so run force_match=False to speed up.


number of objects returned = 1


In [23]:
# Test the df method without force_match
df = bc.get_images_df(imagenames, force_match=False)
print('number of objects returned = {}'.format(len(df)))

number of objects returned = 120


## Feature : lab query

In [4]:
resp = bc._send_message('get', '/labs/')

In [9]:
res= resp.json()['results']

In [11]:
lab = [lab for lab in res if lab['name']=='fermi3'][0]

In [13]:
lab['url']

'http://breadboard-215702.appspot.com/labs/4/'

## Feature : datasets

In [15]:
resp = bc._send_message('get', '/datasets/')

In [16]:
resp.json()

{'count': 3,
 'next': None,
 'previous': None,
 'results': [{'created': '2018-09-07T15:12:43.789917Z',
   'flag': '',
   'id': 1,
   'lab': None,
   'name': '',
   'notes': '',
   'project': None,
   'runs': [],
   'tags': None,
   'url': 'http://breadboard-215702.appspot.com/datasets/1/'},
  {'created': '2018-09-07T15:14:31.868199Z',
   'flag': '',
   'id': 2,
   'lab': None,
   'name': 'DS1',
   'notes': '',
   'project': 2,
   'runs': [],
   'tags': None,
   'url': 'http://breadboard-215702.appspot.com/datasets/2/'},
  {'created': '2018-09-12T01:46:32.135184Z',
   'flag': '',
   'id': 3,
   'lab': None,
   'name': 'test DS 1',
   'notes': '',
   'project': None,
   'runs': [],
   'tags': {},
   'url': 'http://breadboard-215702.appspot.com/datasets/3/'}]}

In [ ]:
# What I want to run:

bc.create_dataset(imagenames, dsname='today/ds1')
df = bc.load_dataset(dsname='june 15/DS1')

"""dsname date options:
- today = today's date
- yesterday = yesterday's date
- MM DD = this year, month, date
- MM DD YY or YYYY 
""" 

# Design choices:

df = bc.load_dataset(dsname='june 15/DS1')

ds = bc.load_dataset(dsname='june 15/DS1')

ds.df = pd.Dataframe...
ds.name = blah
ds.date




In [10]:
import datetime
import re

def timestr_to_datetime(time_string):
    time_string = re.sub(' ','0',time_string[0:19])
    return datetime.datetime.strptime(time_string,'%m-%d-%Y_%H_%M_%S')

def isoformat_datetime(dt_object):
    return dt_object.isoformat()+'Z'

In [11]:
timestr = '10-09-2018_00_21_57_TopA'
dt_object = timestr_to_datetime(timestr)

In [12]:
'2018-10-09T00:21:57Z' == isoformat_datetime(dt_object)

True

In [13]:
type(dt_object)==datetime.datetime

True

In [14]:
type('44')==str

True

In [15]:
TIMEFORMATS = {
    'BEC1': 'blah',
    'FERMI3':'poo'
}

In [16]:
TIMEFORMATS['BEC1']

'blah'

In [7]:
from dateutil import parser

In [8]:
parser.parse('2019-02-27_20-39-28')

ValueError: ('Unknown string format:', '2019-02-27_20-39-28')

In [13]:
resp = client.get_images_json(imagenames)

In [16]:
jsonresp = resp.json()

In [24]:
jsonresp[0]['run']['runtime']

'2018-10-09T00:22:00Z'

In [27]:
import dateutil

In [35]:
int(dateutil.parser.parse('2018-10-09T00:22:00Z').timestamp())

1539044520

In [39]:
pd.__version__

'0.22.0'

In [28]:
for x in tqdm(range(10)):
    print(x)

0
1
2
3
4
5
6
7
8
9

